In [1]:
# %pip install --upgrade openai

In [1]:
import pickle, os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI



In [3]:
from sqlalchemy import create_engine
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
conn_string = (
    "sqlite:///../database/moviesdb.db"
    )

sqlite_engine = create_engine(conn_string, echo=False) #echo can be set to true if you want to see intermediate processing messages as well

print('connection is ok')
print(sqlite_engine.table_names())
db_sql = SQLDatabase(engine=sqlite_engine,
                 sample_rows_in_table_info=2, # We can set the number of samples to be included from database table to help LLM understand the table data better
                )

connection is ok
['movie_characters', 'movie_conversations', 'movie_lines', 'movie_script_urls', 'movie_titles']


C:\Users\hpune\AppData\Local\Temp\ipykernel_34764\2171782936.py:10: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(sqlite_engine.table_names())


In [4]:
index_file_path = '../indexes/langchain_faiss_index_scripts_432.pickle'

#  You can get the pickle files also from - https://drive.google.com/drive/folders/1rWUQbW4gyOvU96Uo8vFSbJH4VTNjAg7g?usp=share_link
with open(index_file_path, 'rb') as handle:
    db_vect = pickle.load(handle)

In [5]:
llm = OpenAI(temperature=0)

In [6]:
retriever = db_vect.as_retriever(search_kwargs={"k": 3}) 
scripts_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [26]:
from langchain.prompts.prompt import PromptTemplate

_NEW_PROMPT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. 
Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. 
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only look for the few relevant columns given the question.
Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which tabl. 
Make sure to always use table_name.column_name full notation in query if there are ambiguous column names in different tables especially in WHERE clause.
Make sure only to answer questions based on given database tables unless specified later explicitly, also provide your sources in result. Generate sql query without quotes. 
Make sure to use wild cards for search queries involving LIKE keyword and make sure to use search queries which are case insensitive
        Use the following format:

        Question: "Question here"
        SQLQuery: "SQL Query to run"
        SQLResult: "Result of the SQLQuery"
        Answer: "Final answer here along with source"

        Only use the tables listed below.
        {table_info}

        Question: {input}
        """

PROMPT = PromptTemplate(
            input_variables=["input", "table_info", "dialect", "top_k"], 
            template=_NEW_PROMPT_TEMPLATE
        )


db_chain = SQLDatabaseChain(llm=llm, database=db_sql, verbose=True, prompt=PROMPT, top_k=5)

In [27]:
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [28]:
tools = [
    Tool(
        name = "Movies SQL Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about movie characters , movie lines, movie ratings for movies available in database ... or if you need to compare two movies based on genre, rating or characters. If error comes try to handle the eror"
    ),
    Tool(
        name = "Movie Scripts Question Answering System",
        func=scripts_qa.run,
        description="useful for when you need to answer questions about a scene or plot of the movie from those which are available in the documents."
    ),  
    Tool(
        name = "Wikipedia external",
        func=db_chain.run,
        description="useful for finding answers if you're not able to find results from either Movie Scripts Question Answering System or from Movies SQL Database"
    ),
]

In [29]:
# # Use below commented code to use a chat conversational agent instead of regular one. For usage, check - https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# llm=ChatOpenAI(temperature=0)
# agent = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [30]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [31]:
agent.run("how many unique characters are there in the movie 10 things i hate about you?")



> Entering new AgentExecutor chain...
 I need to find out how many characters are in the movie
Action: Movies SQL Database
Action Input: 10 things i hate about you

> Entering new SQLDatabaseChain chain...
10 things i hate about you
SQLQuery:
SELECT movie_title, movie_line_text 
FROM movie_titles 
INNER JOIN movie_lines 
ON movie_titles.movie_id = movie_lines.movie_id 
WHERE movie_title LIKE '%10 things i hate about you%' 
LIMIT 5

SQLResult: [('10 things i hate about you', 'They do not!'), ('10 things i hate about you', 'They do to!'), ('10 things i hate about you', 'I hope so.'), ('10 things i hate about you', 'She okay?'), ('10 things i hate about you', "Let's go.")]
Answer:
Some of the lines from the movie 10 things i hate about you are: 
"They do not!", "They do to!", "I hope so.", "She okay?", "Let's go." (Source: movie_titles and movie_lines tables)
> Finished chain.

Observation: 
Some of the lines from the movie 10 things i hate about you are: 
"They do not!", "They do to!",

'There are 8 unique characters in the movie 10 Things I Hate About You.'

In [13]:
agent.run("who is boyfriend of kat in 10 things i hate about you?")



> Entering new AgentExecutor chain...
 I need to find out who Kat's boyfriend is in the movie 10 Things I Hate About You
Action: Movie Scripts Question Answering System
Action Input: who is Kat's boyfriend in 10 Things I Hate About You
Observation:  Patrick Verona
Thought: I now know the final answer
Final Answer: Patrick Verona is Kat's boyfriend in 10 Things I Hate About You.

> Finished chain.


"Patrick Verona is Kat's boyfriend in 10 Things I Hate About You."